БИБЛИОТЕКИ

In [1]:
# библиотеки....
import numpy as np 
import pandas as pd 
import time
import os
import re
from datetime import datetime, timedelta

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ФУНКЦИИ ПАРСИНГА....

In [31]:
# чтение текстового файла...
def read_txt_file(path=str):
  f = open(path, 'r')
  fdata = f.readlines()
  f.close
  return fdata

# получение данных с страницы 8 с атмосферными явлениями
def get_atm_phem(fdata, ap_type, stations_filter):
  
  # список результат
  res_list = []

  # определяем маркеры данных
  page_marker = 'А Т М О С Ф Е Р Н Ы Е   Я В Л Е Н И Я'
  
  # находим индекс строки, с которой начинается страница
  pos = -1
  for i in range(len(fdata)):
    if page_marker in fdata[i]:
      pos = i
      break
  
  # если этой страницы нет, то выход
  if pos == -1:
    return res_list

  # извлечение названия станции, года и месяца
  s = fdata[pos].split()
  s.pop(0)
  n = s.index('Год')
  for i in range(n):
    s.pop(1)
  n = s.index('Месяц')  
  for i in range(n-1):
    s.pop(2)
  s = s[:3]
  st_name = s[0]
  # иногда запятая после станции встречается
  if ',' in st_name:
    st_name = st_name[:len(st_name)-1]
  year = int(s[1])
  month = int(s[2])
  
  # переходим к данным
  pos += 6

  cur_day = 1

  # пока не дойдем до пустой строки
  while fdata[pos] != '\n':

    # день
    try:
      day = int(fdata[pos][2:5])
      cur_day = day
    except:
      pass
    
    # поиск данных о ливнях в строке
    showers = re.findall(ap_type + r'\d{2}.\d{2}-\d{2}.\d{2}', fdata[pos][5:])

    # добавление данных в формате списка ["метеостанция", начало, окончание]
    for shower in showers:

      # час начала события...
      hour_start = int(shower[3:5])
      # синхронизируем час с дискретизацией через 3 часа
      hour_start = 3 * (hour_start // 3)
            
      dt_start = datetime(year, month, cur_day, hour_start, 0, 0) 

      if st_name in stations_filter:
        res_list.append([st_name, dt_start])  

    pos += 1

  return res_list

СПИСОК ИСХОДНЫХ ФАЙЛОВ

In [44]:
path = '/content/drive/My Drive/Bases/WIND_DBASE/Sources'
# рекуррентная функция поиска списка файлов txt в каталоге
file_list = []
def search_txt_files(path):
  for files in os.listdir(path):
    if os.path.isfile(path + '/' + files):
      if files[-4:] == '.txt':
        file_list.append(path + '/' + files)
    elif os.path.isdir(path + '/' + files):
      search_txt_files(path + '/' + files)
# вызов рекуррентного поиска всех txt файлов с занесением их в список file_list
search_txt_files(path)

СЧИТЫВАНИЕ АТМОСФЕРНЫХ ЯВЛЕНИЙ В СЛОВАРЬ

In [81]:
# словарь атмосферных явлений
atm_phen_dict = {'Шквал':            [], 
                 'Метель':           [],
                 'Ливень':           [],
                 'Снег':             [],
                 'Град':             [],
                 'Туман':            [],
                 'Гололед_Изморозь': []} 

# чтение атмомферных явлений для Махачкалы
for file_name in file_list:
  
  # чтение файла в строки
  fdata = read_txt_file(file_name)
  # Шквал
  rab = get_atm_phem(fdata, ap_type='Ш  ', stations_filter=['Махачкала'])
  atm_phen_dict['Шквал'].extend(rab)
  # Метель
  rab = get_atm_phem(fdata, ap_type='МО ', stations_filter=['Махачкала'])
  atm_phen_dict['Метель'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='МН ', stations_filter=['Махачкала'])
  atm_phen_dict['Метель'].extend(rab)
  # Ливень
  rab = get_atm_phem(fdata, ap_type='ДЛ ', stations_filter=['Махачкала'])
  atm_phen_dict['Ливень'].extend(rab)
  # Снег
  rab = get_atm_phem(fdata, ap_type='С  ', stations_filter=['Махачкала'])
  atm_phen_dict['Снег'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='СМ ', stations_filter=['Махачкала'])
  atm_phen_dict['Снег'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='СЛМ', stations_filter=['Махачкала'])
  atm_phen_dict['Снег'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='СЛ ', stations_filter=['Махачкала'])
  atm_phen_dict['Снег'].extend(rab)
  # Град
  rab = get_atm_phem(fdata, ap_type='Гд ', stations_filter=['Махачкала'])
  atm_phen_dict['Град'].extend(rab)
  # Туман
  rab = get_atm_phem(fdata, ap_type='Т  ', stations_filter=['Махачкала'])
  atm_phen_dict['Туман'].extend(rab)
  # Гололед_Изморозь
  rab = get_atm_phem(fdata, ap_type='Гл ', stations_filter=['Махачкала'])
  atm_phen_dict['Гололед_Изморозь'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='Глц', stations_filter=['Махачкала'])
  atm_phen_dict['Гололед_Изморозь'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='Изм', stations_filter=['Махачкала'])
  atm_phen_dict['Гололед_Изморозь'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='ИЗ', stations_filter=['Махачкала'])
  atm_phen_dict['Гололед_Изморозь'].extend(rab)
  rab = get_atm_phem(fdata, ap_type='ИК', stations_filter=['Махачкала'])
  atm_phen_dict['Гололед_Изморозь'].extend(rab)

# статистика записей
for phen in atm_phen_dict:
  print(phen, ', всего считанных записей:', len(atm_phen_dict[phen]))

# примеры записей...
print('\nПримеры записей для Ливней:')
for i in range(5):
  print(atm_phen_dict['Ливень'][i][0], atm_phen_dict['Ливень'][i][1])

Шквал , всего считанных записей: 0
Метель , всего считанных записей: 0
Ливень , всего считанных записей: 1390
Снег , всего считанных записей: 378
Град , всего считанных записей: 1
Туман , всего считанных записей: 360
Гололед_Изморозь , всего считанных записей: 14

Примеры записей для Ливней:
Махачкала 2011-05-01 09:00:00
Махачкала 2011-05-02 15:00:00
Махачкала 2011-05-06 15:00:00
Махачкала 2011-05-06 15:00:00
Махачкала 2011-05-06 00:00:00


СЧИТЫВАЕМ ИСХОДНЫЙ ДАТАФРЕЙМ

In [82]:
df = pd.read_csv('/content/drive/My Drive/Bases/makhachkala_weather_2011_2020_1.csv')
# превращаем колонку datetime в datetime index
df.index = pd.to_datetime(df.datetime)
# Удаляем колонку datetime
df.drop('datetime', axis=1, inplace=True)
df

,t_air,t_soil,t_dew,P_part_water,rel_hum,satur_def,P_atm,precip,wind_dir,wind_speed
datetime,,,,,,,,,,
2011-01-01 00:00:00,6.6,7.0,5.5,9.01,93,0.73,1019.2,0.0,300,3
2011-01-01 03:00:00,6.2,6.0,5.1,8.75,92,0.73,1018.2,0.0,310,4
2011-01-01 06:00:00,5.8,6.0,4.9,8.64,94,0.58,1018.5,0.0,330,3
2011-01-01 09:00:00,5.9,7.0,3.8,8.00,86,1.28,1018.4,0.0,310,6
2011-01-01 12:00:00,5.2,6.0,3.0,7.58,86,1.26,1019.0,0.0,320,6
...,...,...,...,...,...,...,...,...,...,...
2020-12-31 09:00:00,4.3,6.0,1.8,7.01,84,1.33,1026.8,0.0,319,3
2020-12-31 12:00:00,4.7,6.0,2.4,7.29,85,1.29,1026.1,0.0,289,2
2020-12-31 15:00:00,4.4,5.0,2.0,7.09,85,1.25,1025.8,0.0,326,1


ДОБАВЛЯЕМ КОЛОНКИ АТМОСФЕРНЫХ ЯВЛЕНИЙ С ЗАНЕСЕНИЕМ СЧИТАННЫХ ДАННЫХ (ЛИВЕНЬ, СНЕГ, ТУМАН)

In [84]:
# добавляем колонку с ливнями
df['shower'] = 0
# проходимся по всем записям о ливнях и заносим их в эту колонку
for val in atm_phen_dict['Ливень']:
  df.at[val[1], 'shower'] = 1

# добавляем колонку со снегом
df['snow'] = 0
# проходимся по всем записям о ливнях и заносим их в эту колонку
for val in atm_phen_dict['Снег']:
  df.at[val[1], 'snow'] = 1

# добавляем колонку со снегом
df['fog'] = 0
# проходимся по всем записям о ливнях и заносим их в эту колонку
for val in atm_phen_dict['Туман']:
  df.at[val[1], 'fog'] = 1

df

,t_air,t_soil,t_dew,P_part_water,rel_hum,satur_def,P_atm,precip,wind_dir,wind_speed,shower,snow,fog
datetime,,,,,,,,,,,,,
2011-01-01 00:00:00,6.6,7.0,5.5,9.01,93,0.73,1019.2,0.0,300,3,0,0,0
2011-01-01 03:00:00,6.2,6.0,5.1,8.75,92,0.73,1018.2,0.0,310,4,0,0,0
2011-01-01 06:00:00,5.8,6.0,4.9,8.64,94,0.58,1018.5,0.0,330,3,0,0,0
2011-01-01 09:00:00,5.9,7.0,3.8,8.00,86,1.28,1018.4,0.0,310,6,0,0,0
2011-01-01 12:00:00,5.2,6.0,3.0,7.58,86,1.26,1019.0,0.0,320,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 09:00:00,4.3,6.0,1.8,7.01,84,1.33,1026.8,0.0,319,3,0,0,0
2020-12-31 12:00:00,4.7,6.0,2.4,7.29,85,1.29,1026.1,0.0,289,2,0,0,0
2020-12-31 15:00:00,4.4,5.0,2.0,7.09,85,1.25,1025.8,0.0,326,1,0,0,1


In [85]:
# смотрим на распредления ливней
df['shower'].value_counts()

0    28238
1      986
Name: shower, dtype: int64

In [87]:
# смотрим на распредления снегов
df['snow'].value_counts()

0    28895
1      329
Name: snow, dtype: int64

In [88]:
# смотрим на распредления туманов
df['fog'].value_counts()

0    28892
1      332
Name: fog, dtype: int64

ТАК КАК СИЛЬНЫЙ ВЕТЕР В ИСХОДНИКАХ НЕВОЗМОЖНО РАСПАРСИТЬ В БАЗУ, ТО БЕРЕМ КОЛОНКУ СРЕДНЕЙ СКОРОСТИ ВЕТРА ЗА 3 ЧАСА И С ПОМОЩЬЮ ПОРОГА ОПРЕДЕЛЯЕМ СИЛЬНЫЙ ВЕТЕР 

In [89]:
# смотрим на распредления скорости ветра
df['wind_speed'].value_counts()

2     6545
3     5440
1     5223
4     3930
5     2489
6     1643
0     1618
7     1070
8      605
9      346
10     156
11      80
12      47
13      22
14       8
16       1
15       1
Name: wind_speed, dtype: int64

In [90]:
# будем считать, что средний ветер более 7 м/с включает в себя сильные порывы ветра
# создаем колонку сильного ветра

def strong_wind(val):
  if val > 7:
    return 1
  else:
    return 0

df['strong_wind'] = df['wind_speed'].apply(strong_wind)

In [92]:
# смотрим на распредления сильного ветра
df['strong_wind'].value_counts()

0    27958
1     1266
Name: strong_wind, dtype: int64

ТАКИМ ОБРАЗОМ ТАБЛИЦА ПОЛНОСТЬЮ ГОТОВА К ПРОЦЕССУ СОЗДАНИЯ И ОБУЧЕНИЯ МОДЕЛЕЙ ПРОГНОЗА

In [93]:
# сохраняем результат датафрейма в CSV файл
df.to_csv('/content/drive/My Drive/Bases/makhachkala_weather_2011_2020_2.csv')